In [511]:
import csv
import cfbd
import logging
from tqdm import tqdm

class TqdmLoggingHandler(logging.StreamHandler):
    """Avoid tqdm progress bar interruption by logger's output to console"""

    def emit(self, record):
        try:
            msg = self.format(record)
            tqdm.write(msg, end=self.terminator)
        except RecursionError:
            raise
        except Exception:
            self.handleError(record)

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
log.addHandler(TqdmLoggingHandler())

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'Zg3HzWsx+jNtEVi9kgZvsvtOZPy/ZbIakgsgc83IiQ0XjKludC/gEdDSWpg2CcgP'
configuration.api_key_prefix['Authorization'] = 'Bearer'
api_config = cfbd.ApiClient(configuration)
teams_api = cfbd.TeamsApi(api_config)
rankings_api = cfbd.RankingsApi(api_config)
games_api = cfbd.GamesApi(api_config)
stats_api = cfbd.StatsApi(api_config)
betting_api = cfbd.BettingApi(api_config)
metrics_api = cfbd.MetricsApi(api_config)
ratings_api = cfbd.RatingsApi(api_config)
rectruiting_api = cfbd.RecruitingApi(api_config)

In [430]:
line = []
header = []

In [284]:
year = 2022

In [285]:
game = games_api.get_games(year=year, week=7, home='Michigan')

In [286]:
home_elo = game[0].home_pregame_elo
away_elo = game[0].away_pregame_elo
home_points = game[0].home_points
away_points = game[0].away_points
home_team = game[0].home_team
away_team = game[0].away_team
game_id = game[0].id

line.extend([game_id, home_team, away_team, home_points, away_points, home_elo, away_elo])
header.extend(['game_id', ' home_team', 'away_team', 'home_points', 'away_points', 'home_elo', 'away_elo'])

In [288]:
new_header = ['home_talent', 'away_talent']
new_line = [None, None]
for team in talent:
    if team.school == home_team:
        new_line[0] = team.talent
    if team.school == away_team:
        new_line[1] = team.talent
header.extend(new_header)
line.extend(new_line)

In [289]:
home_record = games_api.get_team_records(year=year, team=home_team)[0]
away_record = games_api.get_team_records(year=year, team=away_team)[0]

In [290]:
home_games = home_record.total.games
home_wp = home_record.total.wins/home_record.total.games
away_games = away_record.total.games
away_wp = away_record.total.wins/away_record.total.games
line.extend([home_games, home_wp, away_games, away_wp])
header.extend(['home_games', 'home_wp', 'away_games', 'away_wp'])

In [291]:
home_season_stats = stats_api.get_team_season_stats(year=year, team=home_team)
away_season_stats = stats_api.get_team_season_stats(year=year, team=away_team)

In [436]:
h = {}
a = {}
for i,stat in enumerate(home_season_stats):
    if home_season_stats[i].stat_name in useful_stats:
        h[home_season_stats[i].stat_name] = home_season_stats[i].stat_value
    if away_season_stats[i].stat_name in useful_stats:
        a[away_season_stats[i].stat_name] = away_season_stats[i].stat_value
for key,value in sorted(h.items()):
    header.append('home_'+key)
    line.append(value)
for key,value in sorted(a.items()):
    header.append('away_'+key)
    line.append(value)

In [294]:
home_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=home_team)[0]
away_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=away_team)[0]
hass = home_advanced_season_stats.to_dict()
aass = away_advanced_season_stats.to_dict()

In [295]:
new_line = []
new_header = []
side = ['defense','offense']
for s in side:
    for key,value in sorted(hass[s].items()):
        if type(value) != dict:
            new_header.append("home_"+s+"_"+key)
            new_line.append(value)
        else:
            for key2,value2 in sorted(hass[s][key].items()):
                new_header.append("home_"+s+"_"+key+"_"+key2)
                new_line.append(value2)
line.extend(new_line)
header.extend(new_header)

In [296]:
new_line = []
new_header = []
side = ['defense','offense']
for s in side:
    for key,value in sorted(aass[s].items()):
        if type(value) != dict:
            new_header.append("away_"+s+"_"+key)
            new_line.append(value)
        else:
            for key2,value2 in sorted(aass[s][key].items()):
                new_header.append("away_"+s+"_"+key+"_"+key2)
                new_line.append(value2)
line.extend(new_line)
header.extend(new_header)

In [380]:
year = 2021
games = games_api.get_games(year=year)
talent = teams_api.get_talent(year=year)
record = games_api.get_team_records(year=year)
stats = stats_api.get_team_season_stats(year=year)

In [411]:
games = [g for g in games if g.away_division=='fbs' and g.home_division=='fbs']

In [471]:
#IN ORDER TO FIND LIST OF USEFUL STATS - YEARS FROM 2004 ONWARD ARE USEFUL
l = []
for x in range(2004,2022):
    k = stats_api.get_team_season_stats(year=x, team='Michigan')
    k_tot = sorted([l.stat_name for l in k])
    l.append(k_tot)
stats_list = list(set.intersection(*map(set,l)))

In [506]:
def get_game_data(game):    
    home_elo = game.home_pregame_elo
    away_elo = game.away_pregame_elo
    home_points = game.home_points
    away_points = game.away_points
    home_team = game.home_team
    away_team = game.away_team
    game_id = game.id
    line = [game_id, home_team, away_team, home_points, away_points, home_elo, away_elo]
    header = ['game_id', 'home_team', 'away_team', 'home_points', 'away_points', 'home_elo', 'away_elo']
    return line, header

def get_talent_data(talent, home_team, away_team):
    header = ['home_talent', 'away_talent']
    line = [None, None]
    for team in talent:
        if team.school == home_team:
            line[0] = team.talent
        if team.school == away_team:
            line[1] = team.talent
    return line, header

def get_record_data(record, home_team, away_team):
    for school in record:
        if school.team == home_team:
            home_record = school
    for school in record:
        if school.team == away_team:
            away_record = school
    home_games = home_record.total.games
    home_wp = home_record.total.wins/home_record.total.games
    away_games = away_record.total.games
    away_wp = away_record.total.wins/away_record.total.games
    line = [home_games, home_wp, away_games, away_wp]
    header = ['home_games', 'home_wp', 'away_games', 'away_wp'] 
    return line, header

def get_stat_data(stats, home_team, away_team):
    line = []
    header = []
    home_season_stats = []
    away_season_stats = []
    for stat in stats:
        if stat.team == home_team:
            home_season_stats.append(stat)
        if stat.team == away_team:
            away_season_stats.append(stat)
    h = {}
    a = {}
#     print(home_team, away_team)
#     print(len(home_season_stats), len(away_season_stats))
    if len(home_season_stats)<27 or len(away_season_stats)<27:
        return False, False
    for i,stat in enumerate(home_season_stats):
        if stat.stat_name in stats_list:
            h[stat.stat_name] = stat.stat_value
    for i,stat in enumerate(away_season_stats):
        if stat.stat_name in stats_list:
            a[stat.stat_name] = stat.stat_value
    for key,value in sorted(h.items()):
        header.append('home_'+key)
        line.append(value)
    for key,value in sorted(a.items()):
        header.append('away_'+key)
        line.append(value)
    return line, header

def get_metrics_data(year, home_team, away_team):
    line = []
    header = []
    home_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=home_team)
    away_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=away_team)
#     for attempt in range(10):
#         try:
#             print("Trying:")
#             home_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=home_team)
#             print("Success")
#             break
#         except:
#             print("Failed")
#             if i < 9: # i is zero indexed
#                 continue
#             else:
#                 return False, False
#         print("Unable")
#         return False, False
#     for attempt in range(10):
#         try:
#             print("Trying:")
#             away_advanced_season_stats = stats_api.get_advanced_team_season_stats(year=year, team=away_team)
#             print("Success")
#             break
#         except:
#             print("Failed")
#             if i < 9: # i is zero indexed
#                 continue
#             else:
#                 print("Unable")
#                 return False, False
#         return False, False
    if len(home_advanced_season_stats)==0 or len(away_advanced_season_stats)==0:
        return False, False
    home_advanced_season_stats = home_advanced_season_stats[0]
    away_advanced_season_stats = away_advanced_season_stats[0]
    hass = home_advanced_season_stats.to_dict()
    aass = away_advanced_season_stats.to_dict()
    
    new_line = []
    new_header = []
    side = ['defense','offense']
    for s in side:
        for key,value in sorted(hass[s].items()):
            if type(value) != dict:
                new_header.append("home_"+s+"_"+key)
                new_line.append(value)
            else:
                for key2,value2 in sorted(hass[s][key].items()):
                    new_header.append("home_"+s+"_"+key+"_"+key2)
                    new_line.append(value2)
    line.extend(new_line)
    header.extend(new_header)
    
    new_line = []
    new_header = []
    side = ['defense','offense']
    for s in side:
        for key,value in sorted(aass[s].items()):
            if type(value) != dict:
                new_header.append("away_"+s+"_"+key)
                new_line.append(value)
            else:
                for key2,value2 in sorted(aass[s][key].items()):
                    new_header.append("away_"+s+"_"+key+"_"+key2)
                    new_line.append(value2)
    line.extend(new_line)
    header.extend(new_header)
    return line, header
    
    

In [517]:
for year in range(2022,2023):
    print("Loading "+str(year))
    lines = []
    header = []
    count = 0
    games = games_api.get_games(year=year)
    games = [g for g in games if g.away_division=='fbs' and g.home_division=='fbs']
#     talent = teams_api.get_talent(year=year)
    record = games_api.get_team_records(year=year)
    stats = stats_api.get_team_season_stats(year=year)
    print("Starting "+str(year))
    for game in tqdm(games, bar_format='{l_bar}{bar:70}{r_bar}{bar:-10b}'):
        curr_line = []
        curr_header = []
        #add game data
        game_line, game_header = get_game_data(game)
        curr_line.extend(game_line)
        curr_header.extend(game_header)

        #add talent data
#         talent_line, talent_header = get_talent_data(talent, game.home_team, game.away_team)
#         curr_line.extend(talent_line)
#         curr_header.extend(talent_header)

        #add record data
        record_line, record_header = get_record_data(record, game.home_team, game.away_team)
        curr_line.extend(record_line)
        curr_header.extend(record_header)

        #add season stat data
        stat_line, stat_header = get_stat_data(stats, game.home_team, game.away_team)
        if stat_line == False or stat_header == False:
            continue
        curr_line.extend(stat_line)
        curr_header.extend(stat_header)

        #add advanced metrics data
        metrics_line, metrics_header = get_metrics_data(year, game.home_team, game.away_team)
        if metrics_line == False or metrics_header == False:
            continue
        curr_line.extend(metrics_line)
        curr_header.extend(metrics_header)
        
        lines.append(curr_line)
        if count == 0:
            header = curr_header
        count+=1
    lines.insert(0, header)
    with open("output"+str(year)+".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(lines)
    f.close()
    

Loading 2022
Starting 2022


100%|██████████████████████████████████████████████████████████████████████| 727/727 [52:01<00:00,  4.29s/it]                                                               


In [498]:
lines.insert(0, header)
with open("output2022.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(lines)
f.close()